In [4]:
from random import choice
import numpy as np
import regex as re
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from colorama import Fore, Back, Style

In [675]:
with open('alak4011_clf.p', 'rb') as f:
    clf = pickle.load(f)

In [695]:
class alak:
    def __init__(self, interactive=False, random=False, NN=True, show_mv=True, save_data=False, clf=clf):
        self.random = random
        self.interactive = interactive
        self.play_mode = 'interactive' if self.interactive else 'random'
        self.save_data = save_data
        self.nn = NN
        self.show_mv = show_mv
        self.clf = clf
        self.board = 'xxxx__oooo'
        self.board_arr = []
        self.empty_board = []
        self.o_idx = []
        self.x_idx = []
        self.x = 'x'
        self.o = 'o'
        self.pattern_x = 'ox+o'
        self.pattern_o = 'xo+x'
        self.winner = ''
        self.player1 = ''
        self.player2 = ''
        self.move_data = []
        self.board_data = np.ndarray((0, 10))
        self.move_from = 0
        self.move_to = 0
        self.x_win = 0
        self.o_win = 0
        self.suicide = False
        self.round_count = 0
        self.move_count = 0
        self.gain = 0
    
    def test_remove(self):
        '''
        The tests include, for both sides:
        - simple kill
        - double kill 
        - double kill that involves more than one piece 
        - suicide moves.
        '''

        board_list = ['xoxoxx____', 'xooxooxx__', '__xoo__oxo', '_xoxxxoo____', '_x_oxxoo____',  '_xoxx___x_']
        board_expect = ['x_x_xx____', 'x__x__xx__', '__xoo__o_o', '_xo___oo____', '_x_o__oo____', '_x_xx___x_']
        off_side = ['x', 'x', 'x', 'o', 'o', 'o']
        fail_num = 0
        print("Testing test_remove after move from offensive side:")
        for i, board in enumerate(board_list):
            if self.show_mv: print("\nAfter {:s} move:".format(off_side[i]))
            self.board = board
            if self.show_mv:
                print("Test {:d}".format(i))
                print('Board: ', self.board)
            self.check_kill(off_side[i])
            try:
                assert self.board == board_expect[i]
            except:
                print('i', i)
                fail_num += 1
        tot_num = len(board_list)
        print('{:d} out of {:d} tests passed'.format(len(board_list) - fail_num, len(board_list)))
        if fail_num > 0: print(Fore.RED + '{:d} out of {:d} tests failed'.format(fail_num, len(board_list)))
            
    def move(self, player, p1, p2, board):
        if p1 > p2:
            p1, p2 = p2, p1
            sym1 = player
            sym2 = '_'
        else:
            sym1 = '_'
            sym2 = player
        board = board[0:p1] + sym1 + board[p1 + 1:p2] + sym2 + board[p2 + 1:]
        return board

    def get_board_arr(self):
        self.board_arr = np.array(list(self.board))

    def find(self):
        self.find_xo()
        self.find_empty()

    def find_empty(self):
        self.empty_board = np.where(self.board_arr == '_')[0]

    def find_xo(self):
        self.get_board_arr()
        self.x_idx = np.where(self.board_arr == 'x')[0]
        self.o_idx = np.where(self.board_arr == 'o')[0]

    def print_board(self):
        if self.show_mv: print('{}\n{}'.format(self.board, '0123456789'))

    def remove(self, pattern):
        self.get_board_arr()
        search = re.search(pattern, self.board)

        for i in range(search.start(0) + 1, search.end(0) - 1):
            self.board_arr[i] = '_'
            
        if self.show_mv:
            if (search.start(0) + 1) == (search.end(0) - 2):
                print('\n{} at {} is captured'.format(search.group(0)[1], search.start(0) + 1))
            else:
                print('\n{} from {} to {} are captured'.format(search.group(0)[1], search.start(0) + 1,
                                                               search.end(0) - 2))
        return self.board_arr

    def kill(self, pattern):
        kill = False
        while re.search(pattern, self.board) != None:
            kill = True
            self.board_arr = self.remove(pattern)
            self.board = ''.join([str(elem) for elem in self.board_arr])
            self.print_board()
        return kill

    def save_board(self, board):
        board_status = []
        for i in board:
            if i == 'x':
                board_status.append(1)
            elif i == 'o':
                board_status.append(-1)
            else:
                board_status.append(0)
        return board_status

    def store_move_data(self, move_count):
        for move in range(move_count):
            if self.winner == 'x':
                self.move_data.append(1)
            elif self.winner == 'o':
                self.move_data.append(0)

    def player_turn(self, player, player_idx):
        self.move_player(player, player_idx)
        self.check_kill(player)
        
    def get_match(self, pattern, board):
        return re.search(pattern, board)
    
    def check_kill(self, player):
        if player == self.o:
            if self.kill(self.pattern_x):
                self.gain = 1
                if self.show_mv:
                    print(Fore.BLUE + "Nice move o!")
                    print(Style.RESET_ALL)
            if self.kill(self.pattern_o):
                self.suicide = True
                self.gain = -1
                if self.show_mv:
                    print(Back.RED + "SUICIDE MOVE!")
                    print(Style.RESET_ALL)
        elif player == self.x:
            if self.kill(self.pattern_o):
                self.gain = 1
                if self.show_mv:
                    print(Fore.BLUE + "Nice move x!")
                    print(Style.RESET_ALL)
            if self.kill(self.pattern_x):
                self.suicide = True
                self.gain = -1
                if self.show_mv:
                    print(Back.RED + "SUICIDE MOVE!")
                    print(Style.RESET_ALL)
        if self.show_mv:
            print(Fore.GREEN + 'gain:', self.gain)
            print(Style.RESET_ALL)
        self.gain = 0
        
    def move_player(self, player, player_idx):
        position1 = choice(player_idx)
        position2 = choice(self.empty_board)
        self.board = self.move(player, position1, position2, self.board)
        if self.show_mv: print('\n{} moved from {} to {}'.format(player, position1, position2))
        self.print_board()
        
    def move_user(self, user):
        self.board = self.move(user, self.move_from, self.move_to, self.board)
        self.check_kill(user)
        self.print_board()
    
    def move_comp(self, comp):
        if self.show_mv: print('\n== {} TURN =='.format(comp))
        if self.nn:
            self.player_predict(self.x, self.x_idx) if comp == 'x' else self.player_predict(self.o, self.o_idx)
        else:
            self.player_turn(self.x, self.x_idx) if comp == 'x' else self.player_turn(self.o, self.o_idx)
            
    def player_predict(self, player, player_idx):
        self.move_player_predict(player, player_idx)
        self.check_kill(player)
    
    def predict_move(self, player, player_idx):
        position1 = choice(player_idx)
        position2 = choice(self.empty_board)
        temp_move = self.move(player, position1, position2, self.board)
        temp = np.atleast_2d(self.save_board(temp_move))
        predicted = self.clf.predict(temp)
        #predicted = self.clf.predict_proba(temp)
        return predicted, temp_move, position1, position2

    def move_player_predict(self, player, player_idx):
        predicted, temp_move, position1, position2 = self.predict_move(player, player_idx)
        counter = 0
        pattern = self.pattern_x if player == self.x else self.pattern_o

        #while predicted[0][1] < .9 and counter < len(player_idx) * len(self.empty_board):
        while predicted[0] != 1 and counter < len(player_idx) * len(self.empty_board):
            predicted, temp_move, position1, position2 = self.predict_move(player, player_idx)
            counter += 1
        
        counter = 0
        while self.get_match(pattern, temp_move) and counter < 20:
            predicted, temp_move, position1, position2 = self.predict_move(player, player_idx)
            counter += 1
        if self.show_mv: print('\n{} moved from {} to {}'.format(player, position1, position2))
        self.board = self.move(player, position1, position2, self.board)
        self.print_board()

    def check_vals(self):
        return len(self.o_idx) <= 1 or len(self.x_idx) <= 1

    def get_winner(self):
        if len(self.o_idx) <= 1:
            self.winner = self.x
            self.x_win += 1
        elif len(self.x_idx) <= 1:
            self.winner = self.o
            self.o_win += 1

    def after_win(self):
        self.get_winner()
        self.store_board_data()
        self.store_move_data(len(self.board_data))
        if self.show_mv:
            print(Back.YELLOW + '\nGame over, player {} wins!'.format(self.winner))
            print(Style.RESET_ALL)

    def reset_game(self):
        self.board = 'xxxx__oooo'
        self.board_arr = []
        self.board_data = np.ndarray((0, 10))
        self.empty_board = []
        self.o_idx = []
        self.x_idx = []
        self.suicide = False
        self.round_count = 0
        self.move_count = 0

    def store_board_data(self):
        temp_board = np.array(self.save_board(self.board))
        self.board_data = np.vstack((self.board_data, temp_board))

    def play_round(self, max_round):
        self.print_board()
        for r in range(max_round):
            self.round_count += 1
            if self.save_data:
                if self.round_count >= 15: break
            self.find()
            if self.show_mv:
                print('\nROUND: {}\n============================='.format(self.round_count))
                print('X: {}\nO: {}'.format(len(self.x_idx), len(self.o_idx)))
        
            self.player_turn(self.x, self.x_idx) if self.nn == False else self.player_predict(self.x, self.x_idx)
            # self.player_turn(self.x, self.x_idx)
            self.store_board_data()
            self.move_count += 1
            if self.save_data:
                if self.suicide: break
            self.find_xo()
            if self.check_vals():
                self.after_win()
                break

            self.find()
            # self.player_turn(self.o, self.o_idx) if self.nn == False else self.player_predict(self.o, self.o_idx)
            self.player_turn(self.o, self.o_idx)
            self.store_board_data()
            self.move_count += 1
            if self.save_data:
                if self.suicide: break
            self.find_xo()
            if self.check_vals():
                self.after_win()
                break
                
    def get_input(self, user):
        if self.show_mv: print('== {} TURN ==\n'.format(user))
        self.move_from = self.get_move_from(user, 'Enter move from: ')
        self.move_to = self.get_move_to('Enter move to: ')
        if self.show_mv: print('{} moved from {} to {}'.format(user, self.move_from, self.move_to))
    
    def get_mode(self, prompt):
        while True:
            try:
                answer = input(prompt)
                if answer == '1' or answer == '2':
                    break
                else:
                    print('Enter "1" or "2", try again.')
            except Exception as e:
                print('Invalid entry.')
        return answer
    
    def get_turn(self, prompt):
        while True:
            try:
                answer = input(prompt)
                if answer == 'y' or answer == 'n':
                    break
                else:
                    print('Enter "y" or "n", try again.')
            except Exception as e:
                print('Invalid entry.')
        return answer
    
    def get_move_from(self, user, prompt):
        user_idx = np.where(self.board_arr == user)[0]
        while True:
            try:
                position = int(input(prompt))
                if position in user_idx:
                    break
                else:
                    print('Try again, please select from: {}'.format(user_idx))
            except Exception as e:
                print('Invalid position.')
        return position
    
    def get_move_to(self, prompt):
        self.find_empty()
        while True:
            try:
                position = int(input(prompt))
                if position in self.empty_board: 
                    break
                else:
                    print('Try again, please select from: {}'.format(self.empty_board))
            except Exception as e:
                print('Invalid position.')
        return position
    
    def get_user_player(self, prompt):
        while True:
            try:
                player = input(prompt)
                if player == 'x' or player == 'o':
                    break
                else:
                    print('Enter "x" or "o", try again.')
            except Exception as e:
                print('Invalid entry.')
        return player
    
    def move_interactive(self, player):
        self.move_user(player) if player == self.player1 else self.move_comp(player)
        self.store_board_data()
        self.move_count += 1
        self.find_xo()
        
    def play_interactive(self, max_round):
        print('LET\'S PLAY ALAK GAME!\n-----------------------------')
        game_mode = self.get_mode('Enter "1" to play with computer or "2" for two players: ')
        self.player1 = self.get_user_player('\nChoose stones, enter "x" or "o": ')
        self.player2 = self.x if self.player1 == 'o' else self.o
        go_first = self.get_turn('Do you want to go first? (y/n) ')
        
        self.print_board()
        for r in range(max_round):
            self.round_count += 1
            self.find()
            if self.show_mv:
                print('\nROUND: {}\n============================='.format(self.round_count))
                print('X: {}\nO: {}'.format(len(self.x_idx), len(self.o_idx)))
            
            if go_first == 'y':
                self.get_input(self.player1)
                self.move_interactive(self.player1)
                if self.check_vals():
                    self.after_win()
                    break      
                self.find()
                self.move_interactive(self.player2)
                if self.check_vals():
                    self.after_win()
                    break
            else:
                self.move_interactive(self.player2)
                if self.check_vals():
                    self.after_win()
                    break
                self.get_input(self.player1)
                self.move_interactive(self.player1)
                if self.check_vals():
                    self.after_win()
                    break 
                    
    def play_games(self, n_game):
        max_round = 50
        data = np.ndarray((0, 10))
        for game in range(n_game):
            if self.play_mode == 'interactive':
                self.play_interactive(max_round)
            elif self.play_mode == 'random':
                self.play_round(max_round)
            if not self.save_data:
                data = np.vstack((data, self.board_data))
            else:
                if self.round_count < 15 and self.suicide == False:
                    data = np.vstack((data, self.board_data))
            self.reset_game()
        print(Fore.BLUE + 'x winning rate: {:.2f}%'.format(self.x_win / n_game * 100))
        print('o winning rate: {:.2f}%'.format(self.o_win / n_game * 100))
        print(Style.RESET_ALL)
        return self.move_data, data

In [649]:
with open('alak18872.p', 'wb') as f:
    pickle.dump((move_data, board_data), f)

In [650]:
with open('alak18872.p', 'rb') as f:
    train_data = pickle.load(f)
y, X = train_data
print(len(X))
print(len(y))
print(X.shape)

18872
18872
(18872, 10)


In [651]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.5, random_state=2)

clf2 = MLPClassifier(solver='lbfgs', alpha=1e-10, hidden_layer_sizes=(40, 4), activation='tanh', \
                    random_state=2, max_iter=1000000, learning_rate_init=0.001)
clf2 = clf.fit(X_train, y_train)
print('Training Score:', clf2.score(X_train, y_train))

Training Score: 0.8036244171259008


/Users/marisatania/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:471: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [652]:
predicted = clf2.predict(X_test)
print('Prediction Score:', clf2.score(X_test, y_test))
print("\nFor the classifier, {}".format(clf2, metrics.classification_report(y_test, predicted)))

print("this is the classification report:\n{}\n".format(metrics.classification_report(y_test, predicted)))
print("Confusion matrix:\n{}\n".format(metrics.confusion_matrix(y_test, predicted)))

Prediction Score: 0.7315599830436625

For the classifier, MLPClassifier(activation='tanh', alpha=1e-10, hidden_layer_sizes=(40, 4),
              max_iter=10000000, random_state=2, solver='lbfgs')
this is the classification report:
              precision    recall  f1-score   support

           0       0.71      0.76      0.74      4621
           1       0.76      0.70      0.73      4815

    accuracy                           0.73      9436
   macro avg       0.73      0.73      0.73      9436
weighted avg       0.73      0.73      0.73      9436


Confusion matrix:
[[3529 1092]
 [1441 3374]]



In [653]:
'''save clf'''

with open ('alak18872_clf.p', 'wb') as f:
    pickle.dump(clf2,f)

In [674]:
print(clf.score(board_data, move_data))
predicted = clf.predict(board_data)
print("this is the classification report:\n{}\n".format(metrics.classification_report(move_data, predicted)))
print("Confusion matrix:\n{}\n".format(metrics.confusion_matrix(move_data, predicted)))

0.6836124401913876
this is the classification report:
              precision    recall  f1-score   support

           0       0.52      0.62      0.57       560
           1       0.79      0.72      0.75      1112

    accuracy                           0.68      1672
   macro avg       0.66      0.67      0.66      1672
weighted avg       0.70      0.68      0.69      1672


Confusion matrix:
[[346 214]
 [315 797]]



In [648]:
print(len(move_data))
print(len(board_data))
# print(move_data)
# print(board_data)

18872
18872


In [694]:
a = alak(interactive=False, random=True, NN=True, show_mv=False, save_data=False)
move_data, board_data = a.play_games(100)
# a.test_remove()

x winning rate: 66.00%
o winning rate: 34.00%



In [ ]:
#4011